<a href="https://colab.research.google.com/github/lodi-m/Misinformation-Prone-Communities/blob/main/MPC_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tasks to do: 

* **Further clean both datasets - realizes that left some stuff out which should be cleaned**
*   Get list of questions that going to be looking into 
  * What are the most popular covid-related words? (will need to remove common english words, think there's a library for it, need to look into it)
  * are certain words more used in certain times? try and link to covid events in canada
  * seperate into fake vs real tweets 
     * when are more fake/real tweets tweeted? relate to world events
  * are there any accounts which were specficially just made for the tweet? look at very close tweet date vs user_created date, if within a day then thats likely?
  * 
  * try looking at some common nlp eda questions too

*   Try doing some more complicated data visualization! Look at Omer's EAFP EDA for inspiration? - especially the heatmaps
*   Specifically look into NLP related EDA - is there something specific we can do? Maybe get one of those word map thingys that show which words are most common in all the tweets? ONLY DO THIS FOR THE TESTING SET THOUGH
  * Don't really care about the most common words in training set? Think more about it though 
*   






In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

#### Further Data Cleaning
Looking at both datasets once again, I notice that there are some things which need to be cleaned up, especially the `created_at` and `user_created_at` columns in the training dataset. I will also convert the `label` entries to be integers instead of floats in the training dataset.

I will first start with the training dataset and then move onto the testing dataset.

#### Cleaning training dataset

In [2]:
training_data = pd.read_csv("/content/drive/MyDrive/MPC/notclean_training_data.csv")
training_data

,created_at,id,text,user_created_at,label
0,Mon Apr 27 23:59:45 +0000 2020,1254923219359936512,excellent devastating story anyone who says co...,Wed Jul 01 01:59:30 +0000 2009,0.0
1,Fri May 01 03:33:46 +0000 2020,1256064242572300293,had a grown man today tell me he refuses to w...,Tue Oct 04 23:45:47 +0000 2011,0.0
2,Thu Apr 23 16:39:54 +0000 2020,1253362977828294657,when will you apologize for hawking hydroxycor...,Wed Apr 26 00:38:34 +0000 2017,0.0
3,Thu Apr 23 04:45:54 +0000 2020,1253183292947550208,isamassiveunderstatement covid is just like t...,Sat Nov 09 19:28:24 +0000 2019,0.0
4,Tue Apr 21 16:32:32 +0000 2020,1252636346134355968,just had an encounter with the most ignorant c...,Thu Nov 03 22:36:53 +0000 2016,0.0
...,...,...,...,...,...
19995,Sun Aug 09 07:46:07 +0000 2020,1292366536020127745,what do you thinkpeople who dont take the even...,Tue May 15 10:27:22 +0000 2007,1.0
19996,Wed Apr 01 06:19:35 +0000 2020,1245234333318815744,see that green line at the bottom thats the on...,Sun Jun 21 19:15:21 +0000 2009,1.0
19997,Wed Jul 01 17:23:09 +0000 2020,1278378624392073217,after october when millions join the dole que...,Wed Jul 24 16:13:34 +0000 2013,1.0
19998,Thu Mar 12 19:50:14 +0000 2020,1238190584222961664,to levando pra mim que eles tbm so imunes pld...,Mon Sep 07 17:19:15 +0000 2009,1.0


In [3]:
training_data['date_tweet_created'] = training_data.loc[:, 'created_at'].apply(lambda x : x[0:10] + ' ' + x[25:30])
training_data['time_tweet_created'] = training_data.loc[:, 'created_at'].apply(lambda x : x[11:20])

training_data['date_user_created'] = training_data.loc[:, 'user_created_at'].apply(lambda x : x[0:10] + ' ' + x[25:30])
training_data['time_user_created'] = training_data.loc[:, 'user_created_at'].apply(lambda x : x[11:20])
training_data = training_data.drop(['created_at', 'user_created_at'], axis=1)

In [4]:
training_data['label'] = training_data['label'].astype('int')

In [6]:
training_data.to_csv('final_training_data.csv', index=False)

#### Cleaning testing dataset

Note that I scraped twitter multiple times in order to get my testing data. Here I will be combining all those files and cleaning the final testing dataset as well. 

In [12]:
testing_data1 = pd.read_csv("https://raw.githubusercontent.com/lodi-m/Misinformation-Prone-Communities/main/data/canada_tweets.csv")
testing_data2 = pd.read_csv("/content/drive/MyDrive/MPC/canada_tweets_1.csv")
testing_data = pd.concat([testing_data1, testing_data2])
testing_data

,time_accessed,tweet_id,full_text,created_at,geo,place_name,coordinates,user_location
0,2021-10-22 15:50,1.450000e+18,Bolsanaro has charges against humanity for all...,2021-10-20 15:04,NaN,Surrey,NaN,Turtle Island
1,2021-10-22 15:50,1.450000e+18,Great news: “90% of eligible residents have at...,2021-10-19 11:52,NaN,Ottawa,NaN,"Ottawa, Ontario"
2,2021-10-22 15:50,1.450000e+18,Oregon set to reach herd immunity? \n\nhttps:/...,2021-10-18 11:08,NaN,Toronto,NaN,"Toronto, and Vancouver Canada"
3,2021-10-22 15:50,1.450000e+18,@TorontosMayor maybe you can post some empiric...,2021-10-13 23:14,NaN,Ajax,NaN,"Ajax, Ontario"
4,2021-10-22 15:50,1.450000e+18,Get #vaccinated people - everywhere \nWe need ...,2021-10-22 0:43,NaN,Québec,NaN,"Québec, Canada"
...,...,...,...,...,...,...,...,...
2829,2021-11-21 5:06,1.459050e+18,AUZGMT — Indigenous COVID risk as country reop...,2021-11-12 7:03,"{'type': 'Point', 'coordinates': [49.28248253,...",Vancouver,"{'type': 'Point', 'coordinates': [-123.1272187...",Americas | United Kingdom
2830,2021-11-21 5:06,1.459050e+18,AUZGMT — Push for Indigenous COVID-19 vaccine ...,2021-11-12 7:03,"{'type': 'Point', 'coordinates': [49.28248253,...",Vancouver,"{'type': 'Point', 'coordinates': [-123.1272187...",Americas | United Kingdom
2831,2021-11-21 5:06,1.459050e+18,@vera_tenacious @gingerqueenjen @BarryHunt008 ...,2021-11-12 7:01,NaN,Vancouver,NaN,Vancouver
2832,2021-11-21 5:06,1.459050e+18,@quarantinebeat @vera_tenacious @BarryHunt008 ...,2021-11-12 6:57,NaN,Vancouver,NaN,Vancouver


In [16]:
testing_data['full_text'].duplicated().sum()

625

In [22]:
testing_data.drop_duplicates(subset=['full_text'], inplace=True, ignore_index=True)
testing_data['full_text'].duplicated().sum()

0

In [24]:
testing_data['text'] = testing_data['full_text'].str.lower().str.replace('rt|@[^\s]+|[^A-Za-z ]', '', regex=True)
testing_data['text'] = testing_data['full_text'].str.lower().str.replace('http\S+', '', regex=True) 
testing_data['text'].duplicated().sum()

16

### Exploratory Data Analysis 

As I am working with two seperate datasets, I will be doing EDA for them seperately to keep everything more organized. I will first start with the training dataset and then move onto the testing dataset. All results from both datasets will be summarized and made available at the bottom of this notebook, as well as in a seperate markdown file in the repo. 

---


#### EDA - Training Dataset

The training dataset I am using is from the CoAID (<ins>Co</ins>vid-19 he<ins>A</ins>lthcare m<ins>I</ins>sinformation <ins>D</ins>ataset). You can find the dataset at this repo https://github.com/cuilimeng/CoAID. You can also find more information about how exactly the dataset was created in the research paper located at https://arxiv.org/abs/2006.00885.

#### EDA - Testing Dataset

I created the testing dataset by using the Twitter API. I collected tweets which contained COVID-19 related keywords and were tweeted from anywhere in Canada after January 1st 2019. I disregarded any retweets as well.

I looked at for the following keywords: 
pandemic, covid-19, covid19, covid, vaccine, coronavirus, corona, covidvaccine, mask, facemask, immunization, herd immunity, face mask

You can see exactly how I created the testing dataset at this notebook: https://github.com/lodi-m/Misinformation-Prone-Communities/blob/main/tweet_farming.ipynb 